In [407]:
import requests # Install requests module first.

url = "https://api.coindcx.com/exchange/v1/derivatives/futures/data/active_instruments"

response = requests.get(url)
symbol = response.json()


In [408]:
# symbol

In [409]:
import time
now = time.time()
current_time = int(now)
seconds_in_day = 60 * 60 * 72
start_date = now - (seconds_in_day)


In [410]:
import requests
import talib as ta
import numpy as np


end_time = "n"

def candelstick(symbol,interval):
    url = "https://public.coindcx.com/market_data/candlesticks"
    query_params = {
        "pair": symbol,
        "from": start_date,
        "to": current_time,
        "resolution": interval,  # '1' OR '5' OR '60' OR '1D'
        "pcode": "f"
    }
    response = requests.get(url, params=query_params)
    if response.status_code == 200:
        data = response.json()
        # Process the data as needed
        # print(data)
        data = data['data']
    else:
        print(f"Error: {response.status_code}, {response.text}")

    close = np.array([item['close'] for item in data],dtype=np.double)
    volume = np.array([item['volume'] for item in data],dtype=np.double)
    high = np.array([item['high'] for item in data],dtype=np.double)
    low = np.array([item['low'] for item in data],dtype=np.double)

    upperband,middleband,lowerband = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2, matype=0)

    perc = (((upperband[-2] - lowerband[-2]) / lowerband[-2]) * 100)
    volume_differ = (((volume[-2] - volume[-3]) / volume[-3]) * 100)
    fastk, fastd = ta.STOCHRSI(close, timeperiod=10, fastk_period=5, fastd_period=3, fastd_matype=0)
    rsi_diff = fastk - fastd
    mfi = ta.MFI(high, low, close, volume, timeperiod=10)



    try:
        if close[-2] >= upperband[-2] and volume[-2] < volume[-3] and perc >= 2 and mfi[-2] >= 70:
            print(f"{symbol} sell {perc} & vol {volume_differ} RSI {rsi_diff[-2]} MFI {mfi[-2]} ")

        elif close[-2] <= lowerband[-2] and volume[-2] < volume[-3] and perc >= 2 and  mfi[-2] <= 35:
            print(f"{symbol} buy {perc} {volume_differ} RSI {rsi_diff[-2]} MFI {mfi[-2]}")
    except:
        pass

    

In [411]:
for i in symbol:
    candelstick(i,"15")

B-TIA_USDT sell 4.944486481699166 & vol -33.51703701994605 RSI 16.611878528862903 MFI 73.9124373383889 
B-IMX_USDT sell 2.2648559776460635 & vol -36.18274868471981 RSI 7.105427357601002e-14 MFI 76.95265857336189 


In [412]:
for i in symbol:
    candelstick(i,"30")


In [413]:
for i in symbol:
    candelstick(i,"60")